### Konstantina Andronikou

## This Notebook implements a comparison between Active and Passive

In [1]:
from allennlp_models.pretrained import load_predictor

In [2]:
import checklist
import logging
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect
import csv

In [3]:
from checklist.pred_wrapper import PredictorWrapper

## Active 

In [4]:
logging.getLogger('allennlp.common.params').disabled = True 
logging.getLogger('allennlp.nn.initializers').disabled = True 
logging.getLogger('allennlp.modules.token_embedders.embedding').disabled = True 
logging.getLogger('urllib3.connectionpool').disabled = True 
logging.getLogger('allennlp.common.plugins').disabled = True 
logging.getLogger('allennlp.common.model_card').disabled = True 
logging.getLogger('allennlp.models.archival').disabled = True 
logging.getLogger('allennlp.data.vocabulary').disabled = True 
logging.getLogger('cached_path').disabled = True

srl_predictor = load_predictor('structured-prediction-srl-bert')
output = srl_predictor.predict("Someone drove Mary to school")
output

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'verbs': [{'verb': 'drove',
   'description': '[ARG0: Someone] [V: drove] [ARG1: Mary] [ARG2: to school]',
   'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'B-ARG2', 'I-ARG2']}],
 'words': ['Someone', 'drove', 'Mary', 'to', 'school']}

In [5]:
def predict_srl(data):
    
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred

predict_and_conf = PredictorWrapper.wrap_predict(predict_srl)

In [6]:
d = ["Someone drove Mary to school"]
pred = predict_and_conf(d)
pred

([{'verbs': [{'verb': 'drove',
     'description': '[ARG0: Someone] [V: drove] [ARG1: Mary] [ARG2: to school]',
     'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'B-ARG2', 'I-ARG2']}],
   'words': ['Someone', 'drove', 'Mary', 'to', 'school']}],
 array([1.]))

In [7]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][0]['description']

In [8]:
def get_arg(pred, arg_target='ARG0'):
    predicate_arguments = pred['verbs'][0]
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [9]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [10]:
def found_arg0_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name']])
    arg_0 = get_arg(pred, arg_target='ARG0')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg0_people)

In [11]:
editor = Editor()

# create examples
t = editor.template('{first_name} drove John to school.', meta=True, nsamples=1000, remove_duplicates = True)

#print(type(t))

for k, v in t.items():
    print(k, v)

meta [{'first_name': 'Tim'}, {'first_name': 'Joan'}, {'first_name': 'Dave'}, {'first_name': 'Mark'}, {'first_name': 'Ben'}, {'first_name': 'Kathy'}, {'first_name': 'Sara'}, {'first_name': 'Rose'}, {'first_name': 'Jonathan'}, {'first_name': 'Marilyn'}, {'first_name': 'Sophie'}, {'first_name': 'Ben'}, {'first_name': 'Donald'}, {'first_name': 'Donald'}, {'first_name': 'Fred'}, {'first_name': 'Alan'}, {'first_name': 'Anne'}, {'first_name': 'Robert'}, {'first_name': 'Ryan'}, {'first_name': 'Joan'}, {'first_name': 'Fred'}, {'first_name': 'Ruth'}, {'first_name': 'Emily'}, {'first_name': 'Nancy'}, {'first_name': 'Lauren'}, {'first_name': 'Donna'}, {'first_name': 'Dan'}, {'first_name': 'Bob'}, {'first_name': 'Larry'}, {'first_name': 'Katherine'}, {'first_name': 'Caroline'}, {'first_name': 'Carolyn'}, {'first_name': 'Alexander'}, {'first_name': 'Barbara'}, {'first_name': 'Ashley'}, {'first_name': 'Kevin'}, {'first_name': 'Janet'}, {'first_name': 'Paul'}, {'first_name': 'Francis'}, {'first_name':

In [12]:
with open('Dataset_BERT/Active.txt', 'w') as f:
    print(t.data, file = f)

In [13]:
# initialize a rest object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output_BERT/Active_BERT.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    0 (0.0%)
['Tim', 'drove', 'John', 'to', 'school', '.'] [ True]
['Joan', 'drove', 'John', 'to', 'school', '.'] [ True]
['Dave', 'drove', 'John', 'to', 'school', '.'] [ True]
['Mark', 'drove', 'John', 'to', 'school', '.'] [ True]
['Ben', 'drove', 'John', 'to', 'school', '.'] [ True]
['Kathy', 'drove', 'John', 'to', 'school', '.'] [ True]
['Sara', 'drove', 'John', 'to', 'school', '.'] [ True]
['Rose', 'drove', 'John', 'to', 'school', '.'] [ True]
['Jonathan', 'drove', 'John', 'to', 'school', '.'] [ True]
['Marilyn', 'drove', 'John', 'to', 'school', '.'] [ True]
['Sophie', 'drove', 'John', 'to', 'school', '.'] [ True]
['Ben', 'drove', 'John', 'to', 'school', '.'] [ True]
['Donald', 'drove', 'John', 'to', 'school', '.'] [ True]
['Donald', 'drove', 'John', 'to', 'school', '.'] [ True]
['Fred', 'drove', 'John', 'to', 'school', '.'] [ True]
['Alan', 'drove', 'John', 'to', 'school', '.'] [ True]
['Anne', 'drove', 'John', 'to', 'sch

['Elizabeth', 'drove', 'John', 'to', 'school', '.'] [ True]
['Robin', 'drove', 'John', 'to', 'school', '.'] [ True]
['Albert', 'drove', 'John', 'to', 'school', '.'] [ True]
['Jennifer', 'drove', 'John', 'to', 'school', '.'] [ True]
['Dick', 'drove', 'John', 'to', 'school', '.'] [ True]
['Louise', 'drove', 'John', 'to', 'school', '.'] [ True]
['Jessica', 'drove', 'John', 'to', 'school', '.'] [ True]
['Melissa', 'drove', 'John', 'to', 'school', '.'] [ True]
['Emily', 'drove', 'John', 'to', 'school', '.'] [ True]
['Jessica', 'drove', 'John', 'to', 'school', '.'] [ True]
['Lucy', 'drove', 'John', 'to', 'school', '.'] [ True]
['Ashley', 'drove', 'John', 'to', 'school', '.'] [ True]
['Sam', 'drove', 'John', 'to', 'school', '.'] [ True]
['Martha', 'drove', 'John', 'to', 'school', '.'] [ True]
['Ron', 'drove', 'John', 'to', 'school', '.'] [ True]
['Steve', 'drove', 'John', 'to', 'school', '.'] [ True]
['Annie', 'drove', 'John', 'to', 'school', '.'] [ True]
['Steven', 'drove', 'John', 'to', 'sc

In [14]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
 
original_stdout = sys.stdout  
 
with open('Predictions_BERT/false_sentences_Active_BERT.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 

## PASSIVE

In [15]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][1]['description'] #second verb

In [16]:
def get_arg(pred, arg_target='ARG1'):
    predicate_arguments = pred['verbs'][1]
    #print(predicate_arguments)
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [17]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][1]['description']#second verb
        
    return predicate_structure

In [18]:
def found_arg1_people(x, pred, conf, label=None, meta=None):

    people = set([meta['first_name']])
    arg_1 = get_arg(pred, arg_target='ARG1')

    if arg_1 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg1 = Expect.single(found_arg1_people)

In [19]:
editor = Editor()

# create examples
t = editor.template('{first_name} got driven to school by John.', meta=True, nsamples=1000, remove_duplicates = True)

#print(type(t))

for k, v in t.items():
    print(k, v)

meta [{'first_name': 'Sarah'}, {'first_name': 'Virginia'}, {'first_name': 'Deborah'}, {'first_name': 'Ashley'}, {'first_name': 'Francis'}, {'first_name': 'Jay'}, {'first_name': 'Paul'}, {'first_name': 'Lisa'}, {'first_name': 'Barbara'}, {'first_name': 'Carl'}, {'first_name': 'Cynthia'}, {'first_name': 'Sharon'}, {'first_name': 'Samuel'}, {'first_name': 'Maria'}, {'first_name': 'Amanda'}, {'first_name': 'Jill'}, {'first_name': 'Simon'}, {'first_name': 'Andrea'}, {'first_name': 'Matthew'}, {'first_name': 'Ray'}, {'first_name': 'Jessica'}, {'first_name': 'Rachel'}, {'first_name': 'Nick'}, {'first_name': 'Alice'}, {'first_name': 'Rose'}, {'first_name': 'Alfred'}, {'first_name': 'Mike'}, {'first_name': 'Francis'}, {'first_name': 'Kathryn'}, {'first_name': 'Arthur'}, {'first_name': 'Marilyn'}, {'first_name': 'Ashley'}, {'first_name': 'Paul'}, {'first_name': 'Lucy'}, {'first_name': 'Ben'}, {'first_name': 'Diana'}, {'first_name': 'Bill'}, {'first_name': 'Heather'}, {'first_name': 'Larry'}, {'f

In [20]:
with open('Dataset_BERT/Passive.txt', 'w') as f:
    print(t.data, file = f)

In [21]:
# initialize a rest object
test = MFT(**t, name = 'detect_arg1_name_default_position', expect=expect_arg1)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output_BERT/Passive_BERT.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    0 (0.0%)
['Sarah', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Virginia', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Deborah', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Ashley', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Francis', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Jay', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Paul', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Lisa', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Barbara', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Carl', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Cynthia', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Sharon', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Samuel', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Maria'

In [22]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
 
original_stdout = sys.stdout  
 
with open('Predictions_BERT/false_sentences_Passive_BERT.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 